In [54]:
import unicodedata

import pandas as pd


def remove_accents(input_str):
    # Normalize the string to decompose accents
    nfkd_form = unicodedata.normalize("NFKD", input_str)
    # Keep only characters that are not combining marks
    return "".join(c for c in nfkd_form if not unicodedata.combining(c))


In [55]:
def create_codesystem_mcdts_fsh(row, file):
    # print(row)
    if pd.isna(row["cod_sns"]):
        #    print(row["Nomenclatura"])
        pass
    else:
        code = row["cod_sns"]
        code_conv = row["cod_conv"]
        name = row["Nomenclatura"]
        preco = row["preco"]
        taxa = row["taxa"]
        if " " not in str(code):
            file.write(
                "* #"
                + str(code)
                + '\n    "'
                + str(name).replace('"', '\\"').strip()
                + '"\n    "'
                + str(name).replace('"', '\\"').strip()
                + '"\n'
            )
        # print(preco)
        if not pd.isna(preco):
            try:
                file.write(
                    "  * ^property[+].code = #preco\n  * ^property[=].valueDecimal = "
                    + str(round(preco, 2))
                    + "\n  * ^property[+].code = #taxa\n  * ^property[=].valueDecimal = "
                    + str(round(taxa, 2))
                    + "\n"
                )
            except:
                print("ERRROR", preco)


def create_concept_map_mcdts_fsh(row, file):
    # print(row)
    if pd.isna(row["cod_sns"]):
        # print(row["Nomenclatura"])
        pass
    else:
        code = row["cod_sns"]
        code_conv = row["cod_conv"]

        # print(preco)
        if not pd.isna(code) and not pd.isna(code_conv):
            try:
                file.write(
                    "* group.element[+].code = #"
                    + str(code)
                    + "\n"
                    + "* group.element[=].target.code = #"
                    + str(code_conv)
                    + "\n* group.element[=].target.relationship = #equivalent\n"
                )
            except:
                pass


def process_all_files(LIST, source):
    for area in LIST:
        data = pd.read_excel(source, sheet_name=area, header=None)
        data.columns = ["cod_sns", "cod_conv", "Nomenclatura", "preco", "taxa"]
        name_code = remove_accents(area.replace(" ", "-").lower())
        name_id = remove_accents(area.replace(" ", "").replace("-", "_"))
        name_read = area.capitalize()
        descr_read = data.iloc[0, 0]
        print(descr_read)
        with open("../input/fsh/codesystems/mcdts-" + name_code + ".fsh", "w") as file:
            file.write(
                """// Define a local code system
CodeSystem: MCDTS"""
                + name_id
                + """CS
Id:         mcdts-"""
                + name_code
                + """-cs
Title:     "MCDTS """
                + descr_read.title()
                + """"
Description: \""""
                + descr_read.title()
                + """"

* ^url =  "https://terminologias.hl7.pt/fhir/CodeSystem/mcdts-"""
                + name_code
                + """-cs"
* ^version = "5.9.2024"
* ^author.name = "ACSS"


* ^property[+].code = #preco
* ^property[=].description = "Preço do MCDT"
* ^property[=].type = #decimal
* ^property[+].code = #taxa
* ^property[=].description = "Taxa Moderadora"
* ^property[=].type = #decimal

"""
            )
            row_sums = data.iloc[4:, :].apply(
                create_codesystem_mcdts_fsh, axis=1, file=file
            )  # Axis 1 applies the function row-wise

        with open(
            "../input/fsh/conceptmaps/mcdts-" + name_code + "-sns-conv.fsh", "w"
        ) as file:
            file.write(
                """
Instance: mcdts-"""
                + name_code
                + """-sns-conv
InstanceOf: ConceptMap
Usage: #definition

* version = "5.9.2024"
* name = "MCDTS-"""
                + name_code
                + """-SNS-CONV"
* title = "Mapeamento de códigos SNS para códigos convencionados para """
                + descr_read.title()
                + """ "

* status = #draft
* experimental = true
* date = "2024-12-11"
* publisher = "ACSS / HL7 Portugal"

* group.source = "https://terminologias.hl7.pt/fhir/CodeSystem/mcdts-"""
                + name_code
                + """-cs"
* group.target = "https://www.acss.pt"
                       
"""
            )
            row_sums = data.iloc[4:, :].apply(
                create_concept_map_mcdts_fsh, axis=1, file=file
            )  # Axis 1 applies the function row-wise

In [56]:
LIST = [
    "A-Análises",
    "B-Anatomia",
    "C-Cardiologia",
    "D-M Nuclear",
    "E-EEG",
    "M-Radiologia",
    "F-Gastrenterologia",
    "G-MFR",
    "H-ORL",
    "J-Urologia",
    "I-Pneumologia-Imunoalergologia",
    "L-Neurofisiologia",
]
source = "/Users/joaoalmeida/Downloads/Tabela-MCDT-Convencionados-2024_05.09.2024.xlsx"
process_all_files(LIST, source)

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'A-Análises'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'L-Neurofisiologia'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'N-Consultas'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'O-Psicologia'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {d

TABELA DA ÁREA A - ANÁLISES CLÍNICAS
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
TABELA DA ÁREA B - ANATOMIA PATOLÓGICA
TABELA DA ÁREA C - CARDIOLOGIA


/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'A-Análises'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'L-Neurofisiologia'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'N-Consultas'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'O-Psicologia'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {d

TABELA DA ÁREA D - MEDICINA NUCLEAR
TABELA DA ÁREA E - ELECTROENCEFALOGRAFIA
TABELA DA ÁREA M - TABELA DE RADIOLOGIA 
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
TABELA DA ÁREA F - ENDOSCOPIA GASTRENTEROLÓGICA
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)


/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'A-Análises'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'L-Neurofisiologia'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'N-Consultas'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'O-Psicologia'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {d

TABELA DA ÁREA G - MEDICINA FISICA E DE REABILITAÇÃO
TABELA DA ÁREA H - OTORRINOLARINGOLOGIA
TABELA DA ÁREA J - UROLOGIA
TABELA DA ÁREA I - TABELA DE PNEUMOLOGIA E TABELA DE IMUNOALERGOLOGIA
TABELA DA ÁREA L - NEUROFISIOLOGIA


/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'A-Análises'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'L-Neurofisiologia'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'N-Consultas'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'O-Psicologia'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {d

In [57]:
data = pd.read_excel(source, sheet_name="B-Anatomia", header=None)

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'A-Análises'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'L-Neurofisiologia'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'N-Consultas'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'O-Psicologia'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {d

In [58]:
data

,0,1,2,3,4
0,TABELA DA ÁREA B - ANATOMIA PATOLÓGICA,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,Códigos,NaN,Nomenclatura comum OM/SNS/CONVENCIONADOS,Preço,Taxa Mod.
3,SNS,Conv.,NaN,NaN,NaN
4,NaN,NaN,Para aplicação desta tabela deve consultar-se ...,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,ANATOMIA PATOLÓGICA,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,Diagnóstico por Citopatologia,NaN,NaN
9,30510,008.6,Exame citológico cervico-vaginal,5.42,3
